# The idea here is to model the score (with a poisson response)

In [49]:
library(tidyverse)
library(rpart)
library(pROC)
library(rpart.plot)
library(caret)
library(nnet)
library(randomForest)
library(Matrix)
library(xgboost)
library(zoo)

In [78]:
library(skellam)

In [164]:
df <- read.csv('epl_data_w_features.csv')

In [165]:
df_play <- df %>%
    filter(data_type == 'hist') %>%
    na.omit %>%
    select(-data_type, -id)

In [166]:
logloss <- function(result, team_1_prob, team_2_prob, team_tie_prob){
    
    team_tie <- as.numeric(result == 'tie')
    team_1_win <- as.numeric(result == 'team_1_win')
    team_2_win <- as.numeric(result == 'team_2_win')
    
    log_losses <- team_tie * log(team_tie_prob) + team_1_win * log(team_1_prob) + team_2_win * log(team_2_prob)
    
    log_losses[which(!is.finite(log_losses))] <- NA
    
    return(
         -1 * mean( log_losses, na.rm = T  )
        )
}

In [167]:
holdout <- df_play[round(df_play %>% nrow * 0.95):(df_play %>% nrow),]
df_model <- df_play[1:round(df_play %>% nrow * 0.95),]

In [168]:
index <- caret::createDataPartition(y = df_model$team_1_score, p = 0.8, list = F)

In [169]:
train <- df_model[index,]
test <- df_model[-index,]

## GLM methods

### Standard approach

In [170]:
fit.poiss_1 <- train %>%
                select(-team_2_score, -result) %>%
                glm(formula = team_1_score ~ .,
                       family="poisson")

fit.poiss_2 <- train %>%
                select(-team_1_score, -result) %>%
                glm(formula = team_2_score ~ .,
                       family="poisson")

In [171]:
fit.poiss_2 %>% summary


Call:
glm(formula = team_2_score ~ ., family = "poisson", data = .)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-2.198  -1.223  -0.085   0.517   3.192  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   1.288e+00  9.426e-01   1.366 0.171937    
is_february                  -5.660e-03  1.676e-02  -0.338 0.735632    
is_november                  -1.818e-02  1.761e-02  -1.032 0.301943    
c_ability_3                   2.326e-02  2.754e-01   0.084 0.932668    
d_ability_1                  -2.461e+00  5.288e-01  -4.654 3.26e-06 ***
d_ability_3                   2.521e+00  1.665e+00   1.514 0.129985    
d_ability_4                  -1.757e-01  1.501e+00  -0.117 0.906852    
d_form_4                     -3.646e-02  6.178e-02  -0.590 0.555106    
d_h2h_2                      -9.133e-02  2.646e-02  -3.451 0.000558 ***
team_1_nameAston Villa        6.035e-01  1.650e-01   3.657 0.000255 ***
team_1_nameBirmingha

In [172]:
preds_score_1 <- predict(fit.poiss_1, newdata = test, type="response")
preds_score_2 <- predict(fit.poiss_2, newdata = test, type="response")

In [173]:
test$preds_score_team_1_poiss_standard <- preds_score_1
test$preds_score_team_2_poiss_standard <- preds_score_2

### using the skellam distribution (defined as the difference between two poisson random variables) to infer the match probability

In [174]:
test$prob_team_1 <- pskellam(-1, test$preds_score_team_2_poiss_standard, test$preds_score_team_1_poiss_standard, log.p = F)
test$prob_tie <- dskellam(0, test$preds_score_team_2_poiss_standard, test$preds_score_team_1_poiss_standard, log = F)
test$prob_team_2 <- 1 - pskellam(0, test$preds_score_team_2_poiss_standard, test$preds_score_team_1_poiss_standard, log.p = F)

In [175]:
#test

In [176]:
logloss(test$result, test$prob_tie, test$prob_team_1, test$prob_team_2)

[1] 1.339748

In [177]:
test$team_tie <- as.numeric(test$result == 'tie')
test$team_1_win <- as.numeric(test$result == 'team_1_win')
test$team_2_win <- as.numeric(test$result == 'team_2_win')

In [178]:
pROC::roc(response = test$team_tie, predictor = test$prob_tie)


Call:
roc.default(response = test$team_tie, predictor = test$prob_tie)

Data: test$prob_tie in 653 controls (test$team_tie 0) < 223 cases (test$team_tie 1).
Area under the curve: 0.6078

In [179]:
pROC::roc(response = test$team_1_win, predictor = test$prob_team_1)


Call:
roc.default(response = test$team_1_win, predictor = test$prob_team_1)

Data: test$prob_team_1 in 460 controls (test$team_1_win 0) < 416 cases (test$team_1_win 1).
Area under the curve: 0.7011

In [180]:
pROC::roc(response = test$team_2_win, predictor = test$prob_team_2)


Call:
roc.default(response = test$team_2_win, predictor = test$prob_team_2)

Data: test$prob_team_2 in 639 controls (test$team_2_win 0) < 237 cases (test$team_2_win 1).
Area under the curve: 0.7134

### Results seem good

In [186]:
holdout <- holdout %>%
    filter(!team_1_name %in% c('Brighton','Huddersfield'),
           !team_2_name %in% c('Brighton','Huddersfield'))

In [187]:
preds_score_1 <- predict(fit.poiss_1, newdata = holdout, type="response")
preds_score_2 <- predict(fit.poiss_2, newdata = holdout, type="response")

In [188]:
holdout$preds_score_team_1_poiss_standard <- preds_score_1
holdout$preds_score_team_2_poiss_standard <- preds_score_2

### using the skellam distribution (defined as the difference between two poisson random variables) to infer the match probability

In [190]:
holdout$prob_team_1 <- pskellam(-1, holdout$preds_score_team_2_poiss_standard, holdout$preds_score_team_1_poiss_standard, log.p = F)
holdout$prob_tie <- dskellam(0, holdout$preds_score_team_2_poiss_standard, holdout$preds_score_team_1_poiss_standard, log = F)
holdout$prob_team_2 <- 1 - pskellam(0, holdout$preds_score_team_2_poiss_standard, holdout$preds_score_team_1_poiss_standard, log.p = F)

In [191]:
#test

In [192]:
logloss(holdout$result, holdout$prob_tie, holdout$prob_team_1, holdout$prob_team_2)

[1] 1.408684

In [193]:
holdout$team_tie <- as.numeric(holdout$result == 'tie')
holdout$team_1_win <- as.numeric(holdout$result == 'team_1_win')
holdout$team_2_win <- as.numeric(holdout$result == 'team_2_win')

In [194]:
pROC::roc(response = holdout$team_tie, predictor = holdout$prob_tie)


Call:
roc.default(response = holdout$team_tie, predictor = holdout$prob_tie)

Data: holdout$prob_tie in 172 controls (holdout$team_tie 0) < 50 cases (holdout$team_tie 1).
Area under the curve: 0.5823

In [195]:
pROC::roc(response = holdout$team_1_win, predictor = holdout$prob_team_1)


Call:
roc.default(response = holdout$team_1_win, predictor = holdout$prob_team_1)

Data: holdout$prob_team_1 in 121 controls (holdout$team_1_win 0) < 101 cases (holdout$team_1_win 1).
Area under the curve: 0.7557

In [196]:
pROC::roc(response = holdout$team_2_win, predictor = holdout$prob_team_2)


Call:
roc.default(response = holdout$team_2_win, predictor = holdout$prob_team_2)

Data: holdout$prob_team_2 in 151 controls (holdout$team_2_win 0) < 71 cases (holdout$team_2_win 1).
Area under the curve: 0.7915

### Try boosting on poisson response